Question 4

For the video (problem 1) you have taken, plot the optical flow vectors on each frame using MATLAB’s optical flow codes.
(i) treating every previous frame as a reference frame 
(ii) treating every 11th frame as a reference frame 
(iii) treating every 31st frame as a reference frame

In [1]:
import cv2
import depthai as dai
import numpy as np

import glob
import re 
from pathlib import Path

In [2]:
# function for sorting the video frames into order
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

# taking a frame at a time and appending into img_array
# in numerical order
frame_array = []
for filename in sorted(glob.glob('../video_frames/*.png'), key= numericalSort ):
    frame = cv2.imread(filename)
    frame_array.append(frame)

In [3]:
# ShiTomasi corner detection params
st_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# lucas kanade optical flow params
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Make sure the destination path is present before starting to store the examples
Path(f"./optical_flow/").mkdir(parents=True, exist_ok=True)

In [4]:
'''
    plots the optical flow vectors on each frame
    treating every nth frame as a reference frame
    default: every previous frame (n = 1)
'''

def optical_flow(n=1):
    # Take first frame from frame array and 
    # find good corners in it
    old_gray = cv2.cvtColor(frame_array[0], cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **st_params)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(frame_array[0])

    # for each frame in frame_array, 
    # find corners and compare with previous frame
    for frame in frame_array[::n]:
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        if p1 is not None and st is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # draw the tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
                frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
            img = cv2.add(frame, mask)

            cv2.imshow('frame', img)
            cv2.waitKey(10 + n)

            # Now update the previous frame and previous points
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

            cv2.imwrite("./optical_flow/"+str(n)+".png", img)
            cv2.imwrite("./optical_flow/"+str(n)+"_mask.png", mask)
    cv2.destroyAllWindows()


In [5]:
# treating every previous frame as a reference frame
optical_flow()

In [6]:
# treating every 11th frame as a reference frame
optical_flow(10)

In [7]:
# treating every 31th frame as a reference frame
optical_flow(30)